In [1]:
import warnings
warnings.filterwarnings( 'ignore' )
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, TimeSeriesSplit
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.impute import SimpleImputer
import sys

In [2]:
partition = 100

In [3]:
!{sys.executable} -m pip install pytorch-tabnet --upgrade


[notice] A new release of pip is available: 24.1.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [4]:
trainpath = f'../../../../data/top30groups/LongLatCombined/train1/train{partition}.csv'
testpath = f'../../../../data/top30groups/LongLatCombined/test1/test{partition}.csv'

traindata = pd.read_csv(trainpath, encoding='ISO-8859-1')
testdata = pd.read_csv(testpath, encoding='ISO-8859-1')

In [5]:
import torch
print(torch.cuda.is_available())

True


In [6]:
torch.cuda.empty_cache()

In [7]:
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder

def split_data(dftrain, dftest):
    Xtrain = dftrain.drop(columns=['gname']).values.astype(float)
    Ytrain = dftrain['gname'].values
    Xtest = dftest.drop(columns=['gname']).values.astype(float)
    Ytest = dftest['gname'].values

    le = LabelEncoder()
    Ytrain = le.fit_transform(Ytrain)
    Ytest = le.transform(Ytest)

    #y_pred_decoded = model.label_encoder.inverse_transform(y_pred)
    y_true_decoded = le.inverse_transform(Ytest)

    return Xtrain, Ytrain, Xtest, Ytest, y_true_decoded, le



In [8]:
from sklearn.base import BaseEstimator, ClassifierMixin
import numpy as np
from pytorch_tabnet.tab_model import TabNetClassifier

class TabNetClassifierWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, n_d=8, n_a=8, n_steps=3, gamma=1.3, lambda_sparse=1e-3, optimizer_params=None):
        self.n_d = n_d
        self.n_a = n_a
        self.n_steps = n_steps
        self.gamma = gamma
        self.lambda_sparse = lambda_sparse
        self.optimizer_params = optimizer_params or {'lr': 0.01}
        self.model = None

    def fit(self, X, y):
        self.model = TabNetClassifier(
            n_d=self.n_d,
            n_a=self.n_a,
            n_steps=self.n_steps,
            gamma=self.gamma,
            lambda_sparse=self.lambda_sparse,
            optimizer_params=self.optimizer_params,
            seed=42,
            verbose=0
        )

        self.model.fit(
            X, y,
            eval_set=[(X, y)],
            max_epochs=500,
            patience=120,
            batch_size=1024,
            virtual_batch_size=128,
            eval_metric=['accuracy']
        )

        self.classes_ = np.unique(y)  
        return self

    def predict(self, X):
        return self.model.predict(X)

    def score(self, X, y):
        preds = self.predict(X)
        return (preds == y).mean()


In [9]:
#from pytorch_tabnet.sklearn import TabNetClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor

def find_best_tabnet(Xtrain, Ytrain, n_iter=20):
    print("Starting TabNet grid search")
    print("CUDA available:", torch.cuda.is_available())

    param_dist = {
        'n_d': [8, 16, 24],
        'n_a': [8, 16, 24],
        'n_steps': [3, 4, 5],
        'gamma': [1.0, 1.3, 1.5],
        'lambda_sparse': [1e-4, 1e-3, 1e-2],
        'optimizer_params': [{'lr': 0.01}]
    }

    random_search = RandomizedSearchCV(
        estimator=TabNetClassifierWrapper(),
        param_distributions=param_dist,
        n_iter=n_iter,
        cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
        scoring='accuracy',
        verbose=1,
        n_jobs=-1,
        random_state=42
    )

    random_search.fit(Xtrain, Ytrain)
    print("Best parameters:", random_search.best_params_)
    print("Best accuracy:", random_search.best_score_)

    return random_search.best_params_



In [10]:
from pytorch_tabnet.callbacks import Callback
import time

class EpochTimer(Callback):
    def __init__(self):
        self.epoch_times = []

    def on_epoch_begin(self, epoch_idx, logs=None):
        self.start_time = time.time()

    def on_epoch_end(self, epoch_idx, logs=None):
        duration = time.time() - self.start_time
        self.epoch_times.append(duration)
        print(f"⏱️ Epoch {epoch_idx + 1} took {duration:.2f} seconds")


In [11]:
Xtrain, Ytrain, Xtest, Ytest, Ytest_decoded, le = split_data(traindata, testdata)
best_tabnet_params = find_best_tabnet(Xtrain, Ytrain)

from sklearn.model_selection import train_test_split

# Split 80% train, 20% validation
X_tr, X_val, y_tr, y_val = train_test_split(
    Xtrain, Ytrain,
    test_size=0.2,
    random_state=42,
    stratify=Ytrain
)

# Re-initialize TabNet with best params
final_model = TabNetClassifier(
    **{k: v for k, v in best_tabnet_params.items()},
    verbose=1,
    seed=42
)

epoch_timer = EpochTimer()

# Retrain on full training data
final_model.fit(
    X_tr, y_tr,
    eval_set=[(X_val, y_val)],
    max_epochs=1000,
    patience=200,
    batch_size=1024,
    virtual_batch_size=128,
    callbacks=[epoch_timer]
)



Starting TabNet grid search
CUDA available: True
Fitting 5 folds for each of 20 candidates, totalling 100 fits


Stop training because you reached max_epochs = 500 with best_epoch = 494 and best_val_0_accuracy = 0.97262


/opt/conda/lib/python3.11/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Stop training because you reached max_epochs = 500 with best_epoch = 489 and best_val_0_accuracy = 0.9756


/opt/conda/lib/python3.11/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


...

Stop training because you reached max_epochs = 500 with best_epoch = 477 and best_val_0_accuracy = 0.92619
Stop training because you reached max_epochs = 500 with best_epoch = 489 and best_val_0_accuracy = 0.9


/opt/conda/lib/python3.11/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/opt/conda/lib/python3.11/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Stop training because you reached max_epochs = 500 with best_epoch = 475 and best_val_0_accuracy = 0.94048


/opt/conda/lib/python3.11/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Stop training because you reached max_epochs = 500 with best_epoch = 490 and best_val_0_accuracy = 0.98952
Best parameters: {'optimizer_params': {'lr': 0.01}, 'n_steps': 3, 'n_d': 24, 'n_a': 16, 'lambda_sparse': 0.01, 'gamma': 1.3}
Best accuracy: 0.8876190476190476
epoch 0  | loss: 4.65738 | val_0_accuracy: 0.0119  |  0:00:00s
⏱️ Epoch 1 took 0.04 seconds
epoch 1  | loss: 4.23261 | val_0_accuracy: 0.02619 |  0:00:00s
⏱️ Epoch 2 took 0.04 seconds
epoch 2  | loss: 3.98694 | val_0_accuracy: 0.02143 |  0:00:00s
⏱️ Epoch 3 took 0.03 seconds
epoch 3  | loss: 3.82039 | val_0_accuracy: 0.03571 |  0:00:00s
⏱️ Epoch 4 took 0.04 seconds
epoch 4  | loss: 3.56047 | val_0_accuracy: 0.05476 |  0:00:00s
⏱️ Epoch 5 took 0.04 seconds
epoch 5  | loss: 3.50614 | val_0_accuracy: 0.04524 |  0:00:00s
⏱️ Epoch 6 took 0.04 seconds
epoch 6  | loss: 3.38215 | val_0_accuracy: 0.05476 |  0:00:00s
⏱️ Epoch 7 took 0.03 seconds
epoch 7  | loss: 3.2897  | val_0_accuracy: 0.04524 |  0:00:00s
⏱️ Epoch 8 took 0.04 second

In [12]:
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

file_path = os.path.join("results", f"gtd{partition}.txt")

# Predict class indices for test set
y_pred = final_model.predict(Xtest)
y_proba = final_model.predict_proba(Xtest)
y_pred_decoded = le.inverse_transform(y_pred)
y_true_decoded = le.inverse_transform(Ytest)

# Make sure the directory exists
os.makedirs("results", exist_ok=True)

# Compute accuracy from decoded labels
acc = accuracy_score(y_true_decoded, y_pred_decoded)

# Write metrics to file
with open(file_path, "w") as file:
    file.write(f"Accuracy: {acc:.4f}\n")
    file.write(f"Precision weighted: {precision_score(y_true_decoded, y_pred_decoded, average='weighted'):.4f}\n")
    file.write(f"Recall weighted: {recall_score(y_true_decoded, y_pred_decoded, average='weighted'):.4f}\n")
    file.write(f"F1 Score weighted: {f1_score(y_true_decoded, y_pred_decoded, average='weighted'):.4f}\n")
    file.write(f"Precision micro: {precision_score(y_true_decoded, y_pred_decoded, average='micro'):.4f}\n")
    file.write(f"Recall micro: {recall_score(y_true_decoded, y_pred_decoded, average='micro'):.4f}\n")
    file.write(f"F1 Score micro: {f1_score(y_true_decoded, y_pred_decoded, average='micro'):.4f}\n")
    file.write(f"Precision macro: {precision_score(y_true_decoded, y_pred_decoded, average='macro'):.4f}\n")
    file.write(f"Recall macro: {recall_score(y_true_decoded, y_pred_decoded, average='macro'):.4f}\n")
    file.write(f"F1 Score macro: {f1_score(y_true_decoded, y_pred_decoded, average='macro'):.4f}\n")
    file.write(f"roc auc weighted: {roc_auc_score(y_true_decoded, y_proba, multi_class='ovr', average='weighted'):.4f}\n")
    file.write(f"roc auc macro: {roc_auc_score(y_true_decoded, y_proba, multi_class='ovr', average='macro'):.4f}\n")
    file.write(f"roc auc micro: {roc_auc_score(y_true_decoded, y_proba, multi_class='ovr', average='micro'):.4f}\n")

with open(f"results/epoch_time_gtd{partition}.txt", "w") as f:
    f.write('\n'.join(str(x) for x in epoch_timer.epoch_times))



In [13]:
print(classification_report(Ytest_decoded, y_pred_decoded))

                                                  precision    recall  f1-score   support

                          Abu Sayyaf Group (ASG)       0.96      0.83      0.89        30
        African National Congress (South Africa)       0.97      1.00      0.98        30
                                Al-Qaida in Iraq       0.68      0.83      0.75        30
        Al-Qaida in the Arabian Peninsula (AQAP)       0.84      0.90      0.87        30
                                      Al-Shabaab       1.00      0.90      0.95        30
             Basque Fatherland and Freedom (ETA)       0.94      1.00      0.97        30
                                      Boko Haram       0.79      1.00      0.88        30
  Communist Party of India - Maoist (CPI-Maoist)       0.85      0.97      0.91        30
       Corsican National Liberation Front (FLNC)       0.97      0.97      0.97        30
                       Donetsk People's Republic       0.97      1.00      0.98        30
Farabundo

In [14]:
def plot_confusion_matrix(y_true, y_pred, labels):
    from sklearn.metrics import confusion_matrix
    import matplotlib.pyplot as plt
    import seaborn as sns
    import numpy as np

    cm = confusion_matrix(y_true, y_pred, labels=labels)
    cm_normalized = cm.astype('float') / cm.sum(axis=1, keepdims=True)

    plt.figure(figsize=(18, 16))
    sns.heatmap(cm_normalized,
                annot=True,
                fmt=".2f",
                xticklabels=labels,
                yticklabels=labels,
                cmap="viridis",
                square=True,
                linewidths=0.5,
                cbar_kws={"shrink": 0.8})

    plt.title(f"Normalized Confusion Matrix", fontsize=18)
    plt.xlabel("Predicted Label", fontsize=14)
    plt.ylabel("True Label", fontsize=14)
    plt.xticks(rotation=90)
    plt.yticks(rotation=0)
    plt.tight_layout()

    # Save the figure
    save_path = f"results/confusion_matrix_partition_{partition}.png"
    plt.savefig(save_path, dpi=300)
    plt.close()

    print(f"Saved confusion matrix for partition {partition} to {save_path}")


In [15]:
# Get all unique class labels from the truths
class_labels = np.unique(Ytest_decoded)

plot_confusion_matrix(Ytest_decoded, y_pred_decoded, labels=class_labels)



Saved confusion matrix for partition 100 to results/confusion_matrix_partition_100.png
